In [ ]:
import torch
import os
import sys
sys.path.append("/content")


In [ ]:
rebuild_dataset = False
visualize_sample = True

ZIP_PATH = "/content/drive/MyDrive/datasets/soccertrack.zip"
TARGET_DIR = "/content/drive/MyDrive/datasets/soccertrack"
PREP_PATH = "/content/drive/MyDrive/datasets/soccertrack_prep"

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!nvidia-smi

In [ ]:
# Get RT-DETR
if not os.path.isdir("/content/RT-DETR"):
  %cd /content
  print("Cloning RT-DETR")
  !git clone https://github.com/lyuwenyu/RT-DETR.git
else:
  print("RT-DETR is already cloned.")

%cd RT-DETR/rtdetrv2_pytorch
!pip install --upgrade pip -q
!pip install -r requirements.txt -q
%cd ..

# Get pretrained weights <- starting training from here
if not os.path.isfile("/content/RT-DETR/pretrained/rtdetrv2_r18vd_120e_coco.pth"):
  !mkdir -p pretrained
  !wget -P pretrained https://github.com/lyuwenyu/storage/releases/download/v0.1/rtdetrv2_r18vd_120e_coco.pth

In [ ]:
# Copy and unzip data in GDrive
if not os.path.isdir(TARGET_DIR) and rebuild_dataset:
    print("Unzipping soccertrack dataset into Google Drive ...")
    !unzip -q "$ZIP_PATH" -d "$TARGET_DIR"
else:
    print("Soccertrack dataset already exists, skipping unzip.")

In [ ]:
# Dataset processing
if rebuild_dataset:
    !pip install pandas pillow tqdm opencv-python -q

    from dataset_preparation import (
        extract_frames_from_videos,
        convert_soccertrack_csvs_to_coco
    )

    frame_out_root = f"{PREP_PATH}/frames"
    out_root = f"{PREP_PATH}/coco"

    video_root = f"{TARGET_DIR}/top_view/videos"
    annotation_root = f"{TARGET_DIR}/top_view/annotations"

    train_split = 0.8
    frame_stride = 20

    os.makedirs(PREP_PATH, exist_ok=True)

    if not os.path.isdir(frame_out_root):
        print(f"Extracting frames from {video_root} → {frame_out_root}")
        extract_frames_from_videos(
            video_root=video_root,
            output_root=frame_out_root,
            frame_stride=frame_stride
        )

    if not os.path.isdir(out_root):
        print("Converting SoccerTrack CSVs to COCO format")
        convert_soccertrack_csvs_to_coco(
            annotation_root=annotation_root,
            image_root=frame_out_root,
            output_root=out_root,
            train_split=train_split,
            frame_stride=frame_stride
        )


In [ ]:
if visualize_sample:
    !pip install matplotlib -q

    from visualization import visualize_coco_sample

    coco_root = "/content/drive/MyDrive/datasets/soccertrack_prep/coco"

    visualize_coco_sample(
        coco_json_path=f"{coco_root}/annotations/train.json",
        image_root=f"{coco_root}/images/train"
    )

In [ ]:
# Generate the config file for the dataset
%%bash
cd /content/RT-DETR/rtdetrv2_pytorch
cat << 'EOF' > configs/dataset/soccertrack_coco.yml
task: detection

evaluator:
  type: CocoEvaluator
  iou_types: ['bbox']

num_classes: 2 # Background + Player + ...
remap_mscoco_category: False

train_dataloader:
  type: DataLoader
  dataset:
    type: CocoDetection
    img_folder: /content/drive/MyDrive/datasets/soccertrack_prep/coco/images/train
    ann_file: /content/drive/MyDrive/datasets/soccertrack_prep/coco/annotations/train.json
    return_masks: False
    transforms:
      type: Compose
      ops: ~
  shuffle: True
  num_workers: 2
  drop_last: True
  collate_fn:
    type: BatchImageCollateFunction


val_dataloader:
  type: DataLoader
  dataset:
    type: CocoDetection
    img_folder: /content/drive/MyDrive/datasets/soccertrack_prep/coco/images/val
    ann_file: /content/drive/MyDrive/datasets/soccertrack_prep/coco/annotations/val.json
    return_masks: False
    transforms:
      type: Compose
      ops: ~
  shuffle: False
  num_workers: 2
  drop_last: False
  collate_fn:
    type: BatchImageCollateFunction
EOF


In [ ]:
cfg_path = "/content/RT-DETR/rtdetrv2_pytorch/configs/rtdetrv2/include/dataloader_soccertrack.yml"

# Only write if the file doesn't exist
if not os.path.isfile(cfg_path):
    payload ="""train_dataloader:
  dataset:
    transforms:
      ops:
        - {type: RandomPhotometricDistort, p: 0.5}
        - {type: RandomZoomOut, fill: 0}
        - {type: RandomIoUCrop, p: 0.8}
        - {type: SanitizeBoundingBoxes, min_size: 1}
        - {type: RandomHorizontalFlip}
        - {type: Resize, size: [640, 640], }
        - {type: SanitizeBoundingBoxes, min_size: 1}
        - {type: ConvertPILImage, dtype: 'float32', scale: True}
        - {type: ConvertBoxes, fmt: 'cxcywh', normalize: True}
      policy:
        name: stop_epoch
        epoch: 30 # epoch in [71, ~) stop `ops`
        ops: ['RandomPhotometricDistort', 'RandomZoomOut', 'RandomIoUCrop']

  collate_fn:
    type: BatchImageCollateFunction
    scales: [480, 512, 544, 576, 608, 640, 640, 640, 672, 704, 736, 768, 800]
    stop_epoch: 30 # epoch in [71, ~) stop `multiscales`

  shuffle: True
  total_batch_size: 16 # total batch size equals to 16 (4 * 4)
  num_workers: 2


val_dataloader:
  dataset:
    transforms:
      ops:
        - {type: Resize, size: [640, 640]}
        - {type: ConvertPILImage, dtype: 'float32', scale: True}
  shuffle: False
  total_batch_size: 32
  num_workers: 2
  """

      # Write the payload to file
    with open(cfg_path, "w") as f:
        f.write(payload)

# Verify
!cat /content/RT-DETR/rtdetrv2_pytorch/configs/rtdetrv2/include/dataloader_soccertrack.yml

In [ ]:
cfg_path = "/content/RT-DETR/rtdetrv2_pytorch/configs/rtdetrv2/rtdetrv2_r18vd_120e_soccertrack.yml"

# Only write if the file doesn't exist
if not os.path.isfile(cfg_path):
    payload = """__include__: [
  '../dataset/soccertrack_coco.yml',
  '../runtime.yml',
  './include/dataloader_soccertrack.yml',
  './include/optimizer.yml',
  './include/rtdetrv2_r50vd.yml',
]

output_dir: ./output/rtdetrv2_r18vd_120e_coco

PResNet:
  depth: 18
  freeze_at: -1
  freeze_norm: False
  pretrained: True

HybridEncoder:
  in_channels: [128, 256, 512]
  hidden_dim: 256
  expansion: 0.5

RTDETRTransformerv2:
  num_layers: 3

epoches: 50

optimizer:
  type: AdamW
  params:
    -
      params: '^(?=.*(?:norm|bn)).*$'
      weight_decay: 0.
"""

    # Write the payload to file
    with open(cfg_path, "w") as f:
        f.write(payload)

# Verify
!cat /content/RT-DETR/rtdetrv2_pytorch/configs/rtdetrv2/rtdetrv2_r18vd_120e_soccertrack.yml

In [ ]:
%cd /content/RT-DETR/rtdetrv2_pytorch

!python tools/train.py \
  -c configs/rtdetrv2/rtdetrv2_r18vd_120e_soccertrack.yml \
  --use-amp